## CNN
- Be designed for image classification on the CIFAR-10 dataset

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

### Class

In [2]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)  # Input: 3 channels, Output: 32 channels, Kernel size: 3x3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)  # Pooling layer with 2x2 window
        self.fc1 = nn.Linear(64 * 8 * 8, 512)  # Fully connected layer
        self.fc2 = nn.Linear(512, 10)  # Output layer (10 classes in CIFAR-10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

### Dataset

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False)

100%|███████████████████████████████████████████████████████████████| 170498071/170498071 [01:23<00:00, 2031439.68it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


### Train

In [4]:
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.920
[1,  4000] loss: 1.554
[1,  6000] loss: 1.404
[1,  8000] loss: 1.311
[1, 10000] loss: 1.230
[1, 12000] loss: 1.145
[2,  2000] loss: 1.047
[2,  4000] loss: 1.013
[2,  6000] loss: 0.967
[2,  8000] loss: 0.951
[2, 10000] loss: 0.932
[2, 12000] loss: 0.900
[3,  2000] loss: 0.770
[3,  4000] loss: 0.759
[3,  6000] loss: 0.755
[3,  8000] loss: 0.742
[3, 10000] loss: 0.747
[3, 12000] loss: 0.726
[4,  2000] loss: 0.547
[4,  4000] loss: 0.577
[4,  6000] loss: 0.575
[4,  8000] loss: 0.571
[4, 10000] loss: 0.565
[4, 12000] loss: 0.607
[5,  2000] loss: 0.365
[5,  4000] loss: 0.377
[5,  6000] loss: 0.412
[5,  8000] loss: 0.402
[5, 10000] loss: 0.421
[5, 12000] loss: 0.442
[6,  2000] loss: 0.222
[6,  4000] loss: 0.228
[6,  6000] loss: 0.247
[6,  8000] loss: 0.265
[6, 10000] loss: 0.289
[6, 12000] loss: 0.312
[7,  2000] loss: 0.130
[7,  4000] loss: 0.127
[7,  6000] loss: 0.154
[7,  8000] loss: 0.152
[7, 10000] loss: 0.174
[7, 12000] loss: 0.190
[8,  2000] loss: 0.076
[8,  4000] 

### Test

In [5]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 73 %
